Original code from:
https://colab.research.google.com/github/pytorch/pytorch.github.io/blob/master/assets/hub/pytorch_vision_googlenet.ipynb

In [ ]:
import torch
import copy
# model = torch.hub.load('pytorch/vision:v0.9.0', 'googlenet', pretrained=True)

import torchvision.models as models
model = models.googlenet(pretrained=True)  #this will not pretrain it

model.eval()

In [ ]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [ ]:
from google.colab import files
files.upload()

Saving car1.jpg to car1.jpg


{'car1.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00d\x00d\x00\x00\xff\xdb\x00C\x00\x06\x04\x05\x05\x05\x04\x06\x05\x05\x05\x07\x06\x06\x07\t\x0f\n\t\x08\x08\t\x13\r\x0e\x0b\x0f\x16\x13\x17\x17\x16\x13\x15\x15\x18\x1b#\x1e\x18\x1a!\x1a\x15\x15\x1e)\x1f!$%\'(\'\x18\x1d+.+&.#&\'&\xff\xdb\x00C\x01\x06\x07\x07\t\x08\t\x12\n\n\x12&\x19\x15\x19&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&\xff\xc0\x00\x11\x08\x01H\x02H\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1c\x00\x00\x01\x05\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x02\x03\x05\x06\x07\x00\x01\x08\xff\xc4\x00O\x10\x00\x01\x02\x04\x04\x03\x04\x06\x06\x07\x04\t\x03\x05\x01\x01\x01\x02\x03\x00\x04\x05\x11\x06\x12!1\x13AQ\x07"aq\x14#2R\x81\x91\x15Bb\xa1\xb1\xd13Sr\x82\x92\xc1\xe1\x16$C\xa2\x08\x17%4Dcs\x83\x93T\x94\xb2&5\xc2\xf0\xf1Ed\xff\xc4\x00\x1a\x01\x00\x03\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\xff\xc4\x00)\x11\x00\x02\x02\x02\x02\x01\x03\x03\x05\x01\x01\

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
# input_image = Image.open(filename)
input_image = Image.open('car1.jpg')
# input_image = Image.open('face5.jpg')
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)

In [ ]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2022-02-08 17:27:37--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2022-02-08 17:27:37 (60.6 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [ ]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

sports car 0.32742393016815186
convertible 0.1804104596376419
minivan 0.0995592251420021
beach wagon 0.05115921050310135
pickup 0.033875517547130585




---

Learn the shape of parts in Inception

https://pytorch.org/vision/stable/_modules/torchvision/models/googlenet.html

In [ ]:
model.conv1.conv  #3 channels, 64 filters

In [ ]:
model.conv1.conv.weight.data.shape  # tensor dims of weight matrix (a layer)

In [ ]:
len(model.conv1.conv.weight.data)  #num of filters

In [ ]:
model.conv1.conv.weight.data[0].shape  # dims of a filter

In [ ]:
model.conv1.conv.weight.data[0][0].shape  #dims of a filter's channel



---
Modify weights and run again


Create a copy of the model to compare before + after changes for entire Models

In [ ]:
import copy
new_model = copy.deepcopy(model)

Set entire weight matrix of a SINGLE FILTER to 0. 

'temp_weightMatrix' is Before

'model.conv1.conv.weight.data[0]' is After

In [ ]:
temp_weightMatrix = model.conv1.conv.weight.data[0].detach().clone()

In [ ]:
new_model.conv1.conv.weight.data[0] = new_model.conv1.conv.weight.data[0] * 0  

In [ ]:
temp_weightMatrix  #check that copy by value, not by ref

In [ ]:
new_model.conv1.conv.weight.data[0] 

In [ ]:
# Run again after tensor modification to see how it changed

new_output = new_model(input_batch)
new_probabilities = torch.nn.functional.softmax(new_output[0], dim=0)
new_top5_prob, new_top5_catid = torch.topk(new_probabilities, 5)
for i in range(new_top5_prob.size(0)):
    row = ['before:', categories[top5_catid[i]], round(top5_prob[i].item(), 5), 'after:', categories[new_top5_catid[i]], round(new_top5_prob[i].item(), 5)]
    print("{: >10} {: >15} {: >10} {: >10} {: >15} {: >10}".format(*row))



---


Get feature 4c: 447, a car detector. Set its weights to 0. Before and after on car image.

https://distill.pub/2020/circuits/zoom-in/

In [ ]:
model.inception4c.branch3[1].conv.weight[63].shape

torch.Size([24, 3, 3])

In [ ]:
new_model = copy.deepcopy(model)
new_model.inception4c.branch3[1].conv.weight[63].data = new_model.inception4c.branch3[1].conv.weight[63] * 0
# new_model.inception4c.branch3[1].conv.weight.data = new_model.inception4c.branch3[1].conv.weight * 0

In [ ]:
new_output = new_model(input_batch)
new_probabilities = torch.nn.functional.softmax(new_output[0], dim=0)
new_top5_prob, new_top5_catid = torch.topk(new_probabilities, 5)
for i in range(new_top5_prob.size(0)):
    row = ['before:', categories[top5_catid[i]], round(top5_prob[i].item(), 5), 'after:', categories[new_top5_catid[i]], round(new_top5_prob[i].item(), 5)]
    print("{: >10} {: >15} {: >10} {: >10} {: >15} {: >10}".format(*row))

   before:      sports car    0.32742     after:      sports car    0.48293
   before:     convertible    0.18041     after:     convertible    0.29294
   before:         minivan    0.09956     after:     beach wagon    0.05411
   before:     beach wagon    0.05116     after:          pickup    0.02257
   before:          pickup    0.03388     after:       car wheel    0.02114


---

Get feature 4b: 418, a dog head pose detector. Set its weights to 0. Before and after on dog image.

https://distill.pub/2020/circuits/zoom-in/

In [ ]:
new_model = copy.deepcopy(model)
new_model.inception4b.branch3[1].conv.weight[34].data = new_model.inception4b.branch3[1].conv.weight[34] * 0

In [ ]:
new_output = new_model(input_batch)
new_probabilities = torch.nn.functional.softmax(new_output[0], dim=0)
new_top5_prob, new_top5_catid = torch.topk(new_probabilities, 5)
for i in range(new_top5_prob.size(0)):
    row = ['before:', categories[top5_catid[i]], round(top5_prob[i].item(), 5), 'after:', categories[new_top5_catid[i]], round(new_top5_prob[i].item(), 5)]
    print("{: >10} {: >15} {: >10} {: >10} {: >15} {: >10}".format(*row))

   before:      sports car    0.32742     after:      sports car    0.32742
   before:     convertible    0.18041     after:     convertible    0.18041
   before:         minivan    0.09956     after:         minivan    0.09956
   before:     beach wagon    0.05116     after:     beach wagon    0.05116
   before:          pickup    0.03388     after:          pickup    0.03388




---
Try looping thru and modifying layers


In [ ]:
# # https://stackoverflow.com/questions/54846905/pytorch-get-all-layers-of-model

# named_layers = dict(model.named_modules())
# len(named_layers)
# named_layers.keys()
# # named_layers['conv1']
# # type(model.conv1.conv) == torch.nn.modules.conv.Conv2d # True
# # type(model.conv1) == torch.nn.modules.conv.Conv2d  # False
# # type(named_layers['conv1.conv']) == torch.nn.modules.conv.Conv2d # True

# modifiable_layers = []
# for layer in list(named_layers.values()):  # layer doesn't use ref; it is a copy of the model's layer
#   if type(layer) == torch.nn.modules.conv.Conv2d:
#     modifiable_layers.append(layer)

# modifiable_layers

In [ ]:
new_model = copy.deepcopy(model)
modifiable_layers = []
for layer_name, layer in new_model.named_modules():
  if type(layer) == torch.nn.modules.conv.Conv2d:
    layer.weight.data = layer.weight.data * 0  #call by reference
    modifiable_layers.append(layer)

In [ ]:
#try to find the attribute of layer that contains the name

# for layer in list(dict(new_model.named_modules()).values()):
#     if type(layer) == torch.nn.modules.conv.Conv2d: 
#         break
# dir(layer)
# getattr(layer, '_get_name')

In [ ]:
# modifiable_layers[0].weight.data #ref to original new_model layer

In [ ]:
# new_model.conv2.conv.weight.data  #check call by ref; also 0

In [ ]:
#check after setting all conv2 layer to 0

new_output = new_model(input_batch)
new_probabilities = torch.nn.functional.softmax(new_output[0], dim=0)
new_top5_prob, new_top5_catid = torch.topk(new_probabilities, 5)
for i in range(new_top5_prob.size(0)):
    row = ['before:', categories[top5_catid[i]], round(top5_prob[i].item(), 5), 'after:', categories[new_top5_catid[i]], round(new_top5_prob[i].item(), 5)]
    print("{: >10} {: >15} {: >10} {: >10} {: >15} {: >10}".format(*row))

   before:      sports car    0.32742     after:   walking stick     0.0048
   before:     convertible    0.18041     after:    picket fence     0.0043
   before:         minivan    0.09956     after:             wig    0.00342
   before:     beach wagon    0.05116     after:    green lizard    0.00339
   before:          pickup    0.03388     after:      saltshaker    0.00302




---

Inception seems to be resilient against the change of a filter. Perhaps there are multiple filters that detect dogs. Thus, set entire layers and/or branches to 0.

In [ ]:
new_model = copy.deepcopy(model)
# new_model.conv1.conv.weight.data = new_model.conv1.conv.weight.data * 0
# new_model.conv1.bn.weight.data = new_model.conv1.bn.weight.data * 0
# new_model.conv2.conv.weight.data = new_model.conv2.conv.weight.data * 0
new_model.conv3.conv.weight.data = new_model.conv3.conv.weight.data * 0
# new_model.inception3a.branch1.conv.weight.data[0] = new_model.inception3a.branch1.conv.weight.data[0] * 0
# new_model.inception3a.branch2[0].conv.weight.data[0] = new_model.inception3a.branch2[0].conv.weight.data[0] * 0
# new_model.inception3a.branch2[1].conv.weight.data[0] = new_model.inception3a.branch2[1].conv.weight.data[0] * 0
# new_model.inception3a.branch3[0].conv.weight.data[0] = new_model.inception3a.branch3[0].conv.weight.data[0] * 0
# new_model.inception3a.branch3[1].conv.weight.data[0] = new_model.inception3a.branch3[1].conv.weight.data[0] * 0
# new_model.inception3b.branch1.conv.weight.data[0] = new_model.inception3b.branch1.conv.weight.data[0] * 0

# new_model.inception4b.branch3[1].conv.weight.data = new_model.inception4b.branch3[1].conv.weight.data * 0
# new_model.inception4c.branch3[1].conv.weight.data = new_model.inception4c.branch3[1].conv.weight.data * 0

In [ ]:
# new_model.inception3a.branch2[0]

In [ ]:
new_output = new_model(input_batch)
new_probabilities = torch.nn.functional.softmax(new_output[0], dim=0)
new_top5_prob, new_top5_catid = torch.topk(new_probabilities, 5)
for i in range(new_top5_prob.size(0)):
    row = ['before:', categories[top5_catid[i]], round(top5_prob[i].item(), 5), 'after:', categories[new_top5_catid[i]], round(new_top5_prob[i].item(), 5)]
    print("{: >10} {: >15} {: >10} {: >10} {: >15} {: >10}".format(*row))

   before:         Samoyed    0.93784     after:    space heater     0.4274
   before:      Pomeranian    0.00828     after:     loudspeaker    0.40686
   before:  Great Pyrenees     0.0056     after:   window screen    0.08127
   before:      Arctic fox    0.00553     after:        strainer    0.04828
   before:      white wolf    0.00474     after:           radio    0.00625
